In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import datetime
import seaborn as sns

#import pydicom
import time
from functools import partial
import gc
import operator 
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as D
import torch.nn.functional as F
from sklearn.model_selection import KFold
from tqdm import tqdm, tqdm_notebook
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import warnings
warnings.filterwarnings(action='once')
import pickle
%load_ext autoreload
%autoreload 2
%matplotlib inline
from skimage.io import imread,imshow
from helper import *
import helper
import torchvision.models as models
from torch.optim import Adam
from defenitions import *

/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
/home/reina/anaconda3/envs/RSNA/lib/python3.6/importlib/_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


In [2]:
SEED = 8153
device=device_by_name("Tesla")
#device=device_by_name("RTX")
#device=device_by_name("1060")
torch.cuda.set_device(device)
#device = "cpu"
sendmeemail=Email_Progress(my_gmail,my_pass,to_email,'Densenet161-Copy2-2 results')

In [3]:
def get_submission(test_df,pred):
    epidural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_epidural','Label':torch.sigmoid(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraparenchymal','Label':torch.sigmoid(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_intraventricular','Label':torch.sigmoid(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subarachnoid','Label':torch.sigmoid(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_subdural','Label':torch.sigmoid(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+test_df.PatientID.values+'_any','Label':torch.sigmoid(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [4]:
def get_submission_ids(image_ids,pred,do_sigmoid=True):
    if do_sigmoid:
        func = lambda x:torch.sigmoid(x)
    else:
        func = lambda x:x
    epidural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_epidural','Label':func(pred[:,0])})
    intraparenchymal_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraparenchymal','Label':func(pred[:,1])})
    intraventricular_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_intraventricular','Label':func(pred[:,2])})
    subarachnoid_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subarachnoid','Label':func(pred[:,3])})
    subdural_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_subdural','Label':func(pred[:,4])})
    any_df=pd.DataFrame(data={'ID':'ID_'+image_ids+'_any','Label':func(pred[:,5])}) 
    return pd.concat([epidural_df,
                        intraparenchymal_df,
                        intraventricular_df,
                        subarachnoid_df,
                        subdural_df,
                        any_df]).sort_values('ID').reset_index(drop=True)

In [5]:
old_train_df = pd.read_csv(data_dir+'train.csv')
old_train_df.shape
old_train_df=old_train_df[~old_train_df.PatientID.isin(bad_images)].reset_index(drop=True)
old_train_df=old_train_df.drop_duplicates().reset_index(drop=True)
old_train_df.shape
old_train_df.head()

train_df = pd.read_csv(data_dir+'train_stage2.csv')
train_df.shape


old_test_df = pd.read_csv(data_dir+'test.csv')
old_test_df.head()
assert old_test_df.shape[0]+old_train_df.shape[0]==train_df.shape[0]

n_splits=3
old_split_sid = old_train_df.PID.unique()
old_split_sid.shape[0]
old_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(old_split_sid))


new_split_sid=np.array(list(set(train_df.PID.unique()).difference(old_train_df.PID.unique())))
new_split_sid.shape[0]
new_splits=list(KFold(n_splits=n_splits,shuffle=True, random_state=SEED).split(new_split_sid))
split_sid=np.concatenate([old_split_sid,new_split_sid])
split_sid.shape[0]

splits=[]
for i in range(len(old_splits)):
    splits.append((np.concatenate([old_splits[i][0],new_splits[i][0]+old_split_sid.shape[0]]),
                  np.concatenate([old_splits[i][1],new_splits[i][1]+old_split_sid.shape[0]])))

for i in range(n_splits):
    for j in range(n_splits):
        idx_train = train_df[train_df.PID.isin(set(split_sid[splits[i][0]]))].index.values
        idx_validate =  train_df[train_df.PID.isin(set(split_sid[splits[j][1]]))].index.values
        if i==j:
            print (i,idx_train.shape,idx_validate.shape)
            assert np.unique(np.concatenate([idx_train,idx_validate])).shape[0]==idx_train.shape[0]+idx_validate.shape[0]
        else:
            assert np.unique(np.concatenate([idx_train,idx_validate])).shape[0]==idx_train.shape[0]

(674252, 15)

(674252, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,PID,StudyI,SeriesI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,63eb1e259,0,0,0,0,0,0,a449357f,62d125e5b2,0be5c0d1b3,"['00036', '00036']","['00080', '00080']",180.199951,-125.0,-8.000000
1,2669954a7,0,0,0,0,0,0,363d5865,a20b80c7bf,3564d584db,"['00047', '00047']","['00080', '00080']",922.530821,-156.0,45.572849
2,52c9913b1,0,0,0,0,0,0,9c2b4bd7,3e3634f8cf,973274ffc9,40,150,4.455000,-125.0,-115.063000
3,4e6ff6126,0,0,0,0,0,0,3ae81c2d,a1390c15c2,e5ccad8244,"['00036', '00036']","['00080', '00080']",100.000000,-99.5,28.500000
4,7858edd88,0,0,0,0,0,0,c1867feb,c73e81ed3a,28e0531b3a,40,100,145.793000,-125.0,-132.190000


(752797, 15)

,PatientID,epidural,intraparenchymal,intraventricular,subarachnoid,subdural,any,SeriesI,PID,StudyI,WindowCenter,WindowWidth,ImagePositionZ,ImagePositionX,ImagePositionY
0,28fbab7eb,0.5,0.5,0.5,0.5,0.5,0.5,ebfd7e4506,cf1b6b11,93407cadbb,30,80,158.458000,-125.0,-135.598000
1,877923b8b,0.5,0.5,0.5,0.5,0.5,0.5,6d95084e15,ad8ea58f,a337baa067,30,80,138.729050,-125.0,-101.797981
2,a591477cb,0.5,0.5,0.5,0.5,0.5,0.5,8e06b2c9e0,ecfb278b,0cfe838d54,30,80,60.830002,-125.0,-133.300003
3,42217c898,0.5,0.5,0.5,0.5,0.5,0.5,e800f419cf,e96e31f4,c497ac5bad,30,80,55.388000,-125.0,-146.081000
4,a130c4d2f,0.5,0.5,0.5,0.5,0.5,0.5,faeb7454f3,69affa42,854e4fbc01,30,80,33.516888,-125.0,-118.689819


17079

1859

18938

0 (502659,) (250138,)
1 (501035,) (251762,)
2 (501900,) (250897,)


In [10]:
pickle_file=open(outputs_dir+'PID_splits_3_stage_2.pkl'.format(n_splits),'rb')
split_sid,splits=pickle.load(pickle_file)
pickle_file.close()


In [11]:
def my_loss(y_pred,y_true,weights):
    window=(y_true>=0).to(torch.float)
    loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*weights.expand_as(y_true)).mean()/(window.mean()+1e-7)
    return loss

In [12]:
class Metric():
    def __init__(self,weights,k=0.03):
        self.weights=weights
        self.k=k
        self.zero()
        
    def zero(self):
        self.loss_sum=0.
        self.loss_count=0.
        self.lossf=0.
        
    def calc(self,y_pred,y_true,prefix=""):
        window=(y_true>=0).to(torch.float)
        loss = (F.binary_cross_entropy_with_logits(y_pred,y_true,reduction='none')*window*self.weights.expand_as(y_true)).mean()/(window.mean()+1e-5)
        self.lossf=self.lossf*(1-self.k)+loss*self.k
        self.loss_sum=self.loss_sum+loss*window.sum()
        self.loss_count=self.loss_count+window.sum()
        return({prefix+'mloss':self.lossf})    
        
    def calc_sums(self,prefix=""):
        return({prefix+'mloss_tot':self.loss_sum/self.loss_count})    



In [13]:
#features=(features-features.mean())/features.std()

In [14]:
class SimpleModel(nn.Module):
    def __init__(self,in_size):
        super(SimpleModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (7,in_size), padding=(3,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 5, padding=2)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(64, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [15]:
class SimpleModel2(nn.Module):
    def __init__(self,in_size):
        super(SimpleModel2, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(64, 6, 3, padding=1)
        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [16]:
class ClassModel(nn.Module):
    def __init__(self,in_size):
        super(ClassModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(128)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 128, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(128)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(128, 6, 3, padding=1)
        
        self.conv2d1class=torch.nn.Conv2d(1, 128, (9,in_size), padding=(4,0))
        self.bn0class=torch.nn.BatchNorm1d(128)
        self.maxpool1class=torch.nn.MaxPool1d(3)
        self.conv1d1class=torch.nn.Conv1d(128, 128, 3, padding=1)
        self.bn1class=torch.nn.BatchNorm1d(128)
        self.maxpool2class=torch.nn.MaxPool1d(3)
        self.conv1d2class=torch.nn.Conv1d(128, 64, 2, padding=1)
        self.bn2class=torch.nn.BatchNorm1d(64)

        
        
    def forward(self, x):
        z=x
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = self.relu0(x)
        x = self.conv1d1(x)
        x = self.bn1(x)
        x = self.relu1(x)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x = self.relu2(x)
        z=self.conv2d1class(z.unsqueeze(1)).squeeze(-1)
        z=self.bn0class(z)
        z=self.maxpool1class(z)
        z=self.conv1d1class(z)
        z=self.maxpool2class(z)
        z=self.conv1d2class(z)
        z=self.bn2class(z)
        z=F.max_pool1d(z,kernel_size=z.shape[-1])
        z=z.expand_as(x)
        x=torch.cat([x,z],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [17]:
class ResModel(nn.Module):
    def __init__(self,in_size):
        super(ResModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [18]:
class ResModelPool(nn.Module):
    def __init__(self,in_size):
        super(ResModelPool, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size),stride=(1,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
#        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        
        
    def forward(self, x):
        x=x.unsqueeze(1)
        x = self.conv2d1(x)
        x=F.max_pool2d(x,kernel_size=(1,x.shape[-1])).squeeze(-1)        
        x0 = self.bn0(x)
#        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [19]:
class ResDropModel(nn.Module):
    def __init__(self,in_size,dropout=0.2):
        super(ResDropModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 7, padding=3)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 5, padding=2)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)
        self.dropout=dropout
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.bn0(x)
        x = F.dropout(x,self.dropout)
        x0 = self.relu0(x) 
        x = self.conv1d1(x0)
#        x = self.bn1(x)
        x = F.dropout(x,self.dropout)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
#        x = self.bn2(x)
        x = F.dropout(x,self.dropout)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = F.dropout(x,self.dropout)
        out = self.conv1d3(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [20]:
fn = partial(torch.clamp,min=0,max=1)

In [21]:
class GenReLU(nn.Module):
    def __init__(self,leak=0,add=0,clamp=None):
        super(GenReLU, self).__init__()
        self.leak,self.add=leak,add
        if isinstance(clamp,tuple):
            self.clamp = partial(torch.clamp,min=clamp[0],max=clamp[1])
        elif clamp:
            self.clamp = partial(torch.clamp,min=-clamp,max=clamp)
        else:
            self.clamp=None
            
        
    def forward(self,x):
        x = F.leaky_relu(x,self.leak)
        if self.add:
            x=x+self.add
        if self.clamp:
            x = self.clamp(x)
        return x
        
        

In [22]:

class ResModelIn(nn.Module):
    def __init__(self,in_size):
        super(ResModelIn, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(64, 64, 3, padding=1)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=torch.nn.ReLU()
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=torch.nn.ReLU()
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)

        
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = self.conv1d3(x)
        out = x.transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [23]:

class ResModelInR(nn.Module):
    def __init__(self,in_size):
        super(ResModelInR, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 64, (9,in_size), padding=(4,0))
        self.bn0=torch.nn.BatchNorm1d(64)
        self.relu0= GenReLU(leak=0.01,add=-0.3,clamp=(-1,3))
        self.conv1d1=torch.nn.Conv1d(64, 64, 3, padding=1)
        self.bn1=torch.nn.BatchNorm1d(64)
        self.relu1=GenReLU(leak=0.01,add=-0.3,clamp=(-1,3))
        self.conv1d2=torch.nn.Conv1d(128, 64, 3, padding=1)
        self.bn2=torch.nn.BatchNorm1d(64)
        self.relu2=GenReLU(leak=0,add=0,clamp=(0,2))
        self.conv1d3=torch.nn.Conv1d(192, 6, 3, padding=1)

        
        
    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.conv2d1(x).squeeze(-1)
        x = self.bn0(x)
        x0 = self.relu0(x)
        x = self.conv1d1(x0)
        x = self.bn1(x)
        x1 = self.relu1(x)
        x = torch.cat([x0,x1],1)
        x = self.conv1d2(x)
        x = self.bn2(x)
        x2 = self.relu2(x)
        x = torch.cat([x0,x1,x2],1)
        x = self.conv1d3(x)
        out = x.transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [24]:
class BaseModel(nn.Module):
    def __init__(self):
        super(BaseModel, self).__init__()
        self.dont_do_grad=[]
        self.conv2d1=torch.nn.Conv2d(1, 128, (5,2208), padding=(2,0))
        self.relu0=torch.nn.ReLU()
        self.conv1d1=torch.nn.Conv1d(128, 6, 1)

        
        
    def forward(self, x):
        x = self.conv2d1(x.unsqueeze(1)).squeeze(-1)
        x = self.relu0(x)
        out = self.conv1d1(x).transpose(-1,-2)
        return out 
                    
    def no_grad(self):
        for param in self.parameters():
            param.requires_grad=False

    def do_grad(self):
        for n,p in self.named_parameters():
            p.requires_grad=  not any(nd in n for nd in self.dont_do_grad)


In [25]:
%matplotlib nbagg

for num_split in range(3):
    multi=3
    model_name,version = 'se_resnet101' , 'classifier_splits'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape
    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test',num_split),'rb')
    featurest=pickle.load(pickle_file)
    pickle_file.close()
    featurest.shape
    featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
    featurest.shape
    features=torch.cat([features,featurest[:,:4]])
    features.shape
    dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
    atrain_df=dfm.merge(train_df,on='PatientID')
    
    split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
    print (split_train.shape,split_validate.shape)
    print (train_df[train_df.SeriesI.isin(set(split_train))].shape[0],
          train_df[train_df.SeriesI.isin(set(split_validate))].shape[0],)

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=12
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(atrain_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(atrain_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
    num_train_optimization_steps = num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnet101 classifier_splits 0


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

torch.Size([628360, 256])

torch.Size([78545, 8, 256])

...

{'loss': 0.06259299184953634, 'mloss': tensor(0.0633), 'val_loss': 0.06865190058963093, 'val_mloss_tot': tensor(0.0686)}


{'loss': 0.0617668115446808, 'mloss': tensor(0.0624), 'val_loss': 0.06832055405268561, 'val_mloss_tot': tensor(0.0682)}


{'loss': 0.05871584135697522, 'mloss': tensor(0.0591), 'val_loss': 0.06844604775861762, 'val_mloss_tot': tensor(0.0683)}


{'loss': 0.059152921663200415, 'mloss': tensor(0.0595), 'val_loss': 0.06842021308262396, 'val_mloss_tot': tensor(0.0683)}




0.06832055405268561


In [26]:
%matplotlib nbagg
for num_split in range(3):
    multi=3
    model_name,version = 'se_resnext101_32x4d' , 'classifier_splits'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape
    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test_tta',num_split),'rb')
    featurest=pickle.load(pickle_file)
    pickle_file.close()
    featurest.shape
    featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
    featurest.shape
    features=torch.cat([features,featurest[:,:4]])
    features.shape
    dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
    atrain_df=dfm.merge(train_df,on='PatientID')
    split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
    print (split_train.shape,split_validate.shape)
    print (train_df[train_df.SeriesI.isin(set(split_train))].shape[0],
          train_df[train_df.SeriesI.isin(set(split_validate))].shape[0],)

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=12
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(atrain_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(atrain_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
    num_train_optimization_steps = num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

se_resnext101_32x4d classifier_splits 0


torch.Size([2697008, 256])

torch.Size([674252, 4, 256])

torch.Size([628360, 256])

torch.Size([78545, 8, 256])

...

{'loss': 0.06058445333595275, 'mloss': tensor(0.0612), 'val_loss': 0.06852469546308296, 'val_mloss_tot': tensor(0.0684)}


{'loss': 0.05858686716331698, 'mloss': tensor(0.0583), 'val_loss': 0.06847218390702542, 'val_mloss_tot': tensor(0.0684)}


{'loss': 0.05696543862302366, 'mloss': tensor(0.0574), 'val_loss': 0.06877540232638409, 'val_mloss_tot': tensor(0.0687)}


{'loss': 0.05747396366902702, 'mloss': tensor(0.0584), 'val_loss': 0.06867518262171178, 'val_mloss_tot': tensor(0.0686)}




(0.06813691547593778, {'val_loss': 0.06813691547593778, 'val_mloss_tot': tensor(0.0680)})


In [27]:
%matplotlib nbagg
for num_split in range(3):
    multi=3
    model_name,version = 'Densenet161_3' , 'classifier_splits'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta2',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape
    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test2',num_split),'rb')
    featurest=pickle.load(pickle_file)
    pickle_file.close()
    featurest.shape
    featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
    featurest.shape
    features=torch.cat([features,featurest[:,:4]])
    features.shape
    dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
    atrain_df=dfm.merge(train_df,on='PatientID')
    split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
    print (split_train.shape,split_validate.shape)
    print (train_df[train_df.SeriesI.isin(set(split_train))].shape[0],
          train_df[train_df.SeriesI.isin(set(split_validate))].shape[0],)

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=12
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(atrain_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(atrain_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
    num_train_optimization_steps = num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

Densenet161_3 classifier_splits 0


torch.Size([2697008, 552])

torch.Size([674252, 4, 552])

torch.Size([628360, 552])

torch.Size([78545, 8, 552])

...

{'loss': 0.054690031705979504, 'mloss': tensor(0.0555), 'val_loss': 0.07008900573534842, 'val_mloss_tot': tensor(0.0700)}


{'loss': 0.0561826117285858, 'mloss': tensor(0.0571), 'val_loss': 0.06990268212471125, 'val_mloss_tot': tensor(0.0698)}


{'loss': 0.05885461902145809, 'mloss': tensor(0.0610), 'val_loss': 0.06998016499394996, 'val_mloss_tot': tensor(0.0699)}


{'loss': 0.05924852927349815, 'mloss': tensor(0.0629), 'val_loss': 0.06942898647620681, 'val_mloss_tot': tensor(0.0693)}




(0.06917435775173053, {'val_loss': 0.06917435775173053, 'val_mloss_tot': tensor(0.0690)})


In [28]:
%matplotlib nbagg
for num_split in range(3):
    multi=3
    model_name,version = 'Densenet169_3' , 'classifier_splits'
    print (model_name,version,num_split)
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_train_tta2',num_split),'rb')
    features=pickle.load(pickle_file)
    pickle_file.close()
    features.shape
    features=features.reshape(features.shape[0]//4,4,-1)
    features.shape
    
    pickle_file=open(outputs_dir+outputs_format.format(model_name,version,'features_test2',num_split),'rb')
    featurest=pickle.load(pickle_file)
    pickle_file.close()
    featurest.shape
    featurest=featurest.reshape(featurest.shape[0]//8,8,-1)
    featurest.shape
    features=torch.cat([features,featurest[:,:4]])
    features.shape
    dfm=pd.concat([old_train_df,old_test_df],sort=False,axis=0).reset_index(drop=True)[['PatientID']]
    atrain_df=dfm.merge(train_df,on='PatientID')
    split_train = atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][0]]))].SeriesI.unique()
    split_validate =  atrain_df[atrain_df.PID.isin(set(split_sid[splits[num_split][1]]))].SeriesI.unique()
    print (split_train.shape,split_validate.shape)
    print (train_df[train_df.SeriesI.isin(set(split_train))].shape[0],
          train_df[train_df.SeriesI.isin(set(split_validate))].shape[0],)

    np.random.seed(SEED+num_split)
    torch.manual_seed(SEED+num_split)
    torch.cuda.manual_seed(SEED+num_split)
    torch.backends.cudnn.deterministic = True
    batch_size=16
    num_workers=12
    num_epochs=24
    klr=1
    weights = torch.tensor([1.,1.,1.,1.,1.,2.],device=device)
    train_dataset=FullHeadDataset(atrain_df,
                                  split_train,
                                  features,
                                  'SeriesI',
                                  'ImagePositionZ',
                                  hemorrhage_types,
                                  multi=multi)                
    validate_dataset=FullHeadDataset(atrain_df,
                                     split_validate,
                                     torch.cat([features[:,i,:] for i in range(4)],-1),
                                     'SeriesI',
                                     'ImagePositionZ',
                                     hemorrhage_types)                

    model=ResModelPool(features.shape[-1])
    version=version+'_fullhead_resmodel_pool2_stage2_{}'.format(multi)
    _=model.to(device)
    #mixup=Mixup(device=device)
    loss_func=my_loss
    #fig,ax = plt.subplots(figsize=(10,7))
    #gr=loss_graph(fig,ax,num_epochs,len(train_dataset)//batch_size+1,limits=[0.02,0.06])
    num_train_optimization_steps = num_epochs*(len(train_dataset)//batch_size+int(len(train_dataset)%batch_size>0))
    sched=WarmupExpCosineWithWarmupRestartsSchedule( t_total=num_train_optimization_steps, cycles=2,tau=1)
    optimizer = BertAdam(model.parameters(),lr=klr*1e-3,schedule=sched)
    history,best_model= model_train(model,
                                    optimizer,
                                    train_dataset,
                                    batch_size,
                                    num_epochs,
                                    loss_func,
                                    weights=weights,
                                    do_apex=False,
                                    validate_dataset=validate_dataset,
                                    param_schedualer=None,
                                    weights_data=None,
                                    metric=Metric(torch.tensor([1.,1.,1.,1.,1.,2.])),
                                    return_model=True,
                                    best_average=3,
                                    num_workers=num_workers,
                                    sampler=None,
                                    graph=None)
    torch.save(best_model.state_dict(), models_dir+models_format.format(model_name,version,num_split))

Densenet169_3 classifier_splits 0


torch.Size([2697008, 208])

torch.Size([674252, 4, 208])

torch.Size([628360, 208])

torch.Size([78545, 8, 208])

...

{'loss': 0.06060564276148177, 'mloss': tensor(0.0614), 'val_loss': 0.07034590802633459, 'val_mloss_tot': tensor(0.0702)}


{'loss': 0.059590297119954685, 'mloss': tensor(0.0599), 'val_loss': 0.07027836590675654, 'val_mloss_tot': tensor(0.0702)}


{'loss': 0.061303866701336744, 'mloss': tensor(0.0599), 'val_loss': 0.07045417536767118, 'val_mloss_tot': tensor(0.0703)}


{'loss': 0.06016535607166075, 'mloss': tensor(0.0617), 'val_loss': 0.07047930070347425, 'val_mloss_tot': tensor(0.0704)}




0.0702729734934174
